In [243]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2 
import seaborn as sns
from sqlalchemy import create_engine, inspect


In [244]:
# PostgreSQL credentials
DB_HOST = "localhost"  
DB_PORT = "5432"
DB_NAME = "truck_data_db"
DB_USER = "postgres"
DB_PASSWORD = "saibaba"

# Create engine for PostgreSQL connection
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# Use SQLAlchemy Inspector to get table names dynamically
inspector = inspect(engine)
tables = inspector.get_table_names()

dataframes = {}

# Load each table into a pandas DataFrame
for table in tables:
    # Use pandas to read the table into a DataFrame
    dataframes[table] = pd.read_sql_table(table, con=engine)

    print(f"Exported {table} ")

Exported city_weather 
Exported drivers_table 
Exported routes_table 
Exported routes_weather 
Exported traffic_table 
Exported truck_schedule_table 
Exported trucks_table 


In [ ]:
print("truck_schedule_table: ",dataframes['truck_schedule_table'].columns)
print("trucks_table: ",dataframes['trucks_table'].columns)

print("drivers_table: ",dataframes['drivers_table'].columns)

print("routes_table: ",dataframes['routes_table'].columns)

print("routes_weather: ",dataframes['routes_weather'].columns)

print("traffic_table: ",dataframes['traffic_table'].columns)

print("city_weather: ",dataframes['city_weather'].columns)




Merging tables:

In [245]:
df = pd.merge(dataframes['truck_schedule_table'],dataframes['trucks_table'],on='truck_id',how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   truck_id              12308 non-null  int64  
 1   route_id              12308 non-null  object 
 2   departure_date        12308 non-null  object 
 3   estimated_arrival     12308 non-null  object 
 4   delay                 12308 non-null  int64  
 5   truck_age             12308 non-null  int64  
 6   load_capacity_pounds  11704 non-null  float64
 7   mileage_mpg           12308 non-null  int64  
 8   fuel_type             11917 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 865.5+ KB


In [246]:
df = pd.merge(df,dataframes['drivers_table'],left_on="truck_id", right_on="vehicle_no",how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   truck_id              12308 non-null  int64  
 1   route_id              12308 non-null  object 
 2   departure_date        12308 non-null  object 
 3   estimated_arrival     12308 non-null  object 
 4   delay                 12308 non-null  int64  
 5   truck_age             12308 non-null  int64  
 6   load_capacity_pounds  11704 non-null  float64
 7   mileage_mpg           12308 non-null  int64  
 8   fuel_type             11917 non-null  object 
 9   driver_id             12308 non-null  object 
 10  name                  12308 non-null  object 
 11  gender                12074 non-null  object 
 12  age                   12308 non-null  int64  
 13  experience            12308 non-null  int64  
 14  driving_style         11831 non-null  object 
 15  ratings            

In [247]:
df = pd.merge(df,dataframes['routes_table'],on='route_id',how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   truck_id              12308 non-null  int64  
 1   route_id              12308 non-null  object 
 2   departure_date        12308 non-null  object 
 3   estimated_arrival     12308 non-null  object 
 4   delay                 12308 non-null  int64  
 5   truck_age             12308 non-null  int64  
 6   load_capacity_pounds  11704 non-null  float64
 7   mileage_mpg           12308 non-null  int64  
 8   fuel_type             11917 non-null  object 
 9   driver_id             12308 non-null  object 
 10  name                  12308 non-null  object 
 11  gender                12074 non-null  object 
 12  age                   12308 non-null  int64  
 13  experience            12308 non-null  int64  
 14  driving_style         11831 non-null  object 
 15  ratings            

routes_weather table merging

In [248]:
df['departure_date'] = pd.to_datetime(df['departure_date'], errors='coerce')
dataframes['routes_weather']['Date'] = pd.to_datetime(dataframes['routes_weather']['Date'], errors='coerce')


In [249]:
df.sort_values(by='departure_date', inplace=True)
dataframes['routes_weather'].sort_values(by='Date', inplace=True)
df = pd.merge_asof(
    df,
    dataframes['routes_weather'],
    by='route_id',                 # Ensure route_id is the same on both DataFrames
    left_on='departure_date',
    right_on='Date',
    direction='backward'           # Merge with backward direction
)

# Check the result
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   truck_id              12308 non-null  int64         
 1   route_id              12308 non-null  object        
 2   departure_date        12308 non-null  datetime64[ns]
 3   estimated_arrival     12308 non-null  object        
 4   delay                 12308 non-null  int64         
 5   truck_age             12308 non-null  int64         
 6   load_capacity_pounds  11704 non-null  float64       
 7   mileage_mpg           12308 non-null  int64         
 8   fuel_type             11917 non-null  object        
 9   driver_id             12308 non-null  object        
 10  name                  12308 non-null  object        
 11  gender                12074 non-null  object        
 12  age                   12308 non-null  int64         
 13  experience      

city_weather merging 

In [ ]:
dataframes['city_weather'] = dataframes['city_weather'].add_prefix('city_')
dataframes['city_weather'].info()



In [252]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
dataframes['city_weather']['city_date'] = pd.to_datetime(dataframes['city_weather']['city_date'], errors='coerce')

# Extract only the date part
df['Date'] = df['Date'].dt.date
dataframes['city_weather']['city_date'] = dataframes['city_weather']['city_date'].dt.date

In [256]:
# Filter the city_weather DataFrame where the hour column equals 700
filtered_city_weather = dataframes['city_weather'][dataframes['city_weather']['city_hour'] == 700]
filtered_city_weather = filtered_city_weather.drop_duplicates(subset=['city_city_id', 'city_date'])

# Perform the merge
df = pd.merge(
    df,
    filtered_city_weather,
    left_on=['origin_id', 'Date'],
    right_on=['city_city_id', 'city_date'],
    how='left'
)
print(df.info())


In [258]:
dataframes['city_weather']['combined_datetime'] = pd.to_datetime(dataframes['city_weather']['city_date'].astype(str) + ' ' + dataframes['city_weather']['city_hour'].astype(str).str.zfill(4), format='%Y-%m-%d %H%M')

In [260]:
df['estimated_arrival'] = pd.to_datetime(df['estimated_arrival'],errors='coerce')
df['estimated_arrival'] = df['estimated_arrival'].dt.round('H')

0       2019-01-01 13:13:12
1       2019-01-05 17:08:24
2       2019-01-05 23:11:24
3       2019-01-05 23:18:36
4       2019-01-06 21:06:00
                ...        
12303   2019-02-13 06:15:36
12304   2019-02-12 21:28:12
12305   2019-02-12 15:25:12
12306   2019-02-12 09:15:36
12307   2019-02-13 01:06:36
Name: estimated_arrival, Length: 12308, dtype: datetime64[ns]


In [262]:
# Perform the merge
df = pd.merge(
    df,
    dataframes['city_weather'],
    left_on=['destination_id', 'estimated_arrival'],
    right_on=['city_city_id', 'combined_datetime'],
    how='left'
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12602 entries, 0 to 12601
Data columns (total 63 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   truck_id                12602 non-null  int64         
 1   route_id                12602 non-null  object        
 2   departure_date          12602 non-null  datetime64[ns]
 3   estimated_arrival       12602 non-null  datetime64[ns]
 4   delay                   12602 non-null  int64         
 5   truck_age               12602 non-null  int64         
 6   load_capacity_pounds    11980 non-null  float64       
 7   mileage_mpg             12602 non-null  int64         
 8   fuel_type               12200 non-null  object        
 9   driver_id               12602 non-null  object        
 10  name                    12602 non-null  object        
 11  gender                  12360 non-null  object        
 12  age                     12602 non-null  int64 

Traffic_table merging:

In [235]:
dataframes['traffic_table'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2597913 entries, 0 to 2597912
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   route_id        object        
 1   date            datetime64[ns]
 2   hour            int64         
 3   no_of_vehicles  float64       
 4   accident        int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 99.1+ MB


In [ ]:
dataframes['traffic_table']['combined_datetime'] = pd.to_datetime(dataframes['traffic_table']['date'].astype(str) + ' ' + dataframes['traffic_table']['hour'].astype(str).str.zfill(4), format='%Y-%m-%d %H%M')

print(dataframes['traffic_table']['combined_datetime'])
print(df['departure_date'])

0         2019-01-01 00:00:00
1         2019-01-01 01:00:00
2         2019-01-01 02:00:00
3         2019-01-01 03:00:00
4         2019-01-01 04:00:00
                  ...        
2597908   2019-02-15 19:00:00
2597909   2019-02-15 20:00:00
2597910   2019-02-15 21:00:00
2597911   2019-02-15 22:00:00
2597912   2019-02-15 23:00:00
Name: combined_datetime, Length: 2597913, dtype: datetime64[ns]
0       2019-01-01 07:00:00
1       2019-01-01 07:00:00
2       2019-01-01 07:00:00
3       2019-01-01 07:00:00
4       2019-01-01 07:00:00
                ...        
12597   2019-02-12 07:00:00
12598   2019-02-12 07:00:00
12599   2019-02-12 07:00:00
12600   2019-02-12 07:00:00
12601   2019-02-12 07:00:00
Name: departure_date, Length: 12602, dtype: datetime64[ns]


In [ ]:
df = pd.merge(df, 
              dataframes['traffic_table'], 
              left_on=['route_id', 'departure_date'],
              right_on=['route_id','combined_datetime'],
              how='left')  

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12607 entries, 0 to 12606
Data columns (total 68 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   truck_id                12607 non-null  int64         
 1   route_id                12607 non-null  object        
 2   departure_date          12607 non-null  datetime64[ns]
 3   estimated_arrival       12607 non-null  datetime64[ns]
 4   delay                   12607 non-null  int64         
 5   truck_age               12607 non-null  int64         
 6   load_capacity_pounds    11985 non-null  float64       
 7   mileage_mpg             12607 non-null  int64         
 8   fuel_type               12205 non-null  object        
 9   driver_id               12607 non-null  object        
 10  name                    12607 non-null  object        
 11  gender                  12365 non-null  object        
 12  age                     12607 non-null  int64 

In [267]:
df.to_csv('combineddf.csv', index=False)

DONE